In [3]:
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *
import pandas as pd
from ngsolve import grad as oldgrad

#### Configuration Setup

In [5]:
beta = (2,0.001)
eps = 0.01

p = lambda x: x + (exp(beta[0]*(x-1)/eps)-exp(-beta[0]/eps))/(exp(-beta[0]/eps)-1)
q = lambda y: y + (exp(beta[1]*(y-1)/eps)-exp(-beta[1]/eps))/(exp(-beta[1]/eps)-1)

exact = p(x) * q(y)
coeff =  beta[1] * p(x) +  beta[0] * q(y)


new_config = {
    'method': 'hdg', #options are dg, hdg
    'order': [1, 2, 3],
    'beta': (2,0.001),
    'mesh_size': [1.0, 0.5, 0.25, 0.1250, 0.0625, 0.0313],
    'epsilon': 0.01,
    'exact': exact,
    'coeff': coeff,
    'alpha': 20, 
    'enrich': True, 
    'enrich_functions':[p(x), q(y)],
    'enrich_domain_ind':[lambda x,y,h: x > 1 - h/2, lambda x,y,h: y > 1 - h/2]
}


#### Enrichment Proxy Functions for the DG method

In [7]:
class EnrichmentProxy(CoefficientFunction):
    """
    Provide wrappers for grad/Other and multiplication of enrichment lists for the DG Method.
    """
    def __init__(self, func, enr_list):
        self.func = func
        self.enr_list = enr_list
        self.grad_list = [CoefficientFunction((coeff.Diff(x), coeff.Diff(y))) for coeff in self.enr_list ]

    def __call__(self):
        return self.func[0] + sum([self.func[i]*self.enr_list[i-1] for i in range(1,len(self.enr_list)+1)])

    def __mul__(self, other):
        if type(other) == EnrichmentProxy:
            return self() * other()
        else:
            return self() * other
    
    def __rmul__(self,other):
        return self.__mul__(other)    
    
    def __add__(self, other):
        if type(other) == EnrichmentProxy:
            return self() + other()
        else:
            return self() + other
    
    def __radd__(self,other):
        return self.__add__(other)
    
    def __sub__(self, other):
        if type(other) == EnrichmentProxy:
            return self() - other()
        else:
            return self() - other
    def __rsub__(self,other):
        return self.__sub__(other)
    
    '''gradient of newly enriched approximation spaces'''
    def grad(self):
        mygrad = oldgrad(self.func[0])
        for i in range(1,len(self.enr_list)):
            mygrad += self.func[i] * self.grad_list[i-1]
            mygrad += oldgrad(self.func[i])*self.enr_list[i-1]
        return mygrad
    
    '''define proxy functions for Other'''
    def Other(self):
        return EnrichmentProxy([f.Other() for f in self.func],self.enr_list)

#### Enrichment Proxy Functions for the HDG method

In [9]:
class EnrichmentProxy_VOL(CoefficientFunction):
    """
    Provide wrappers for grad/Other and multiplication of enrichment lists for the HDG method.
    """
    def __init__(self, func, enr_list):
        self.func = func
        self.enr_list = enr_list
        self.grad_list = [CoefficientFunction((coeff.Diff(x), coeff.Diff(y))) for coeff in self.enr_list ]
    
    def __call__(self):
        return self.func[0] + sum([self.func[i]*self.enr_list[i-1] for i in range(1,len(self.enr_list)+1)])

    def __sub__(self, facet):
        return self() - facet
    
    def __rsub__(self,facet):
        return self.__sub__(facet)

    def __mul__(self, facet):
        return self() * facet
    
    def __rmul__(self,facet):
        return self.__mul__(facet)    
    
    def __add__(self, facet):
        return self() + facet
    
    def __radd__(self,facet):
        return self.__add__(facet)

    '''gradient of newly enriched approximation spaces'''
    def grad(self):
        mygrad = oldgrad(self.func[0])
        for i in range(1,len(self.enr_list)):
            mygrad += self.func[i] * self.grad_list[i-1]
            mygrad += oldgrad(self.func[i])*self.enr_list[i-1]
        return mygrad

def grad(q):
    q.grad()


In [11]:
class EnrichmentProxy_FAC(CoefficientFunction):
    """
    Provide wrappers for for facets.
    """
    def __init__(self, func, enr_list):
        self.func = func
        self.enr_list = enr_list
        self.grad_list = [CoefficientFunction((coeff.Diff(x), coeff.Diff(y))) for coeff in self.enr_list ]
    
    def __call__(self):
        return self.func[3] + sum([self.func[3 + i]*self.enr_list[i-1] for i in range(3,len(self.enr_list)+1)])

    def __sub__(self, facet):
        return self() - facet
    
    def __rsub__(self,facet):
        return self.__sub__(facet)

    def __mul__(self, facet):
        return self() * facet
    
    def __rmul__(self,facet):
        return self.__mul__(facet)    
    
    def __add__(self, facet):
        return self() + facet
    
    def __radd__(self,facet):
        return self.__add__(facet)

#### Functions to mark/select elements to be enriched

In [13]:
'''Mark elements in the mesh to be enriched'''
def mark_element(Q, mesh, enr_indicator, mesh_size):
    ba = BitArray(Q.ndof)        
    ba.Clear()
    for el in Q.Elements():
        mark = False
        for v in el.vertices:
            if (enr_indicator(mesh[v].point[0],mesh[v].point[1],mesh_size)):
                mark = True
        for dof in el.dofs:
            ba[dof] = mark
        Qx = Compress(Q, active_dofs=ba)     
    return Qx


'''Mark elements boundaries (facets) in the mesh to be enriched'''
def mark_element_bnd(Q, QF, mesh, enr_indicator, mesh_size):
    ba = BitArray(Q.ndof)        
    ba.Clear()
    for el in Q.Elements():
        mark = False
        for v in el.vertices:
            if (enr_indicator(mesh[v].point[0],mesh[v].point[1],mesh_size)):
                mark = True
        for dof in el.dofs:
            ba[dof] = mark

    gfF = GridFunction(QF)     
    gfF.vec[:] = 0
    for el in QF.Elements():
        if ba[el.nr]:
            for dof in el.dofs:
                gfF.vec[dof] += 1
    ba_F = BitArray(QF.ndof)
    ba_F.Clear()
    for i in range(QF.ndof):
        if gfF.vec[i] > 1.5:
            ba_F[i] = True
    QFx = Compress(QF, active_dofs=ba_F) 
    return QFx

In [15]:
"""
(Hybrid) Discontinuous Galerkin Method with Upwinding
"""

class Convection_Diffusion():
    
    '''Specify default parameters if necessary'''
    config = {
    }
    
    def __init__(self, new_config={}):
        self.config.update(new_config) #Updates the configuration parameters with the user input        
    
    '''Solves the convection-diffusion equations by the 
    DG methods with the options to either enrich or not'''
    def _solveDG(self):
        for order in self.config['order']:
            for size in self.config['mesh_size']:
                mesh = Mesh(unit_square.GenerateMesh(maxh=size))
                V = L2(mesh, order=order, dgjumps=True)        
                Vlist = [V]
                Q = L2(mesh, order=0)
                    
                for enr_indicator in self.config['enrich_domain_ind']:
                    Vlist.append(mark_element(Q,mesh,enr_indicator,size))
                fes = FESpace(Vlist, dgjumps = True)
                u = EnrichmentProxy(fes.TrialFunction(), self.config['enrich_functions'])
                v = EnrichmentProxy(fes.TestFunction(), self.config['enrich_functions'])

                jump_u = u-u.Other()
                jump_v = v-v.Other()

                n = specialcf.normal(2)
                mean_dudn = 0.5 * n * (grad(u) + grad(u.Other()))
                mean_dvdn = 0.5 * n * (grad(v) + grad(v.Other()))

                h = specialcf.mesh_size

                # diffusion
                diffusion = grad(u) * grad(v) * dx \
                    +self.config['alpha'] * order ** 2/ h * jump_u * jump_v * dx(skeleton=True) \
                    +(-mean_dudn * jump_v - mean_dvdn * jump_u) * dx(skeleton=True) \
                    +self.config['alpha'] * order ** 2/h * u * v * ds(skeleton=True) \
                    + (-n * grad(u) * v -n * grad(v) * u) * ds(skeleton=True)

                # convection
                b = CoefficientFunction((self.config['beta'][0], self.config['beta'][1]))
                uup = IfPos(b * n, u(), u.Other()())
                convection = -b * u * grad(v) * dx + b * n * uup * jump_v * dx(skeleton=True)

                acd = BilinearForm(fes)
                acd += self.config['epsilon'] * diffusion + convection
                acd.Assemble()

                # rhs
                f = LinearForm(fes)
                f += SymbolicLFI(self.config['coeff'] * v)
                f.Assemble()

                gfu = GridFunction(fes, name="uDG")
                
                ## added the try-catch since numerical factorization failed 
                try:
                    gfu.vec.data = acd.mat.Inverse(freedofs=fes.FreeDofs(),inverse="umfpack") * f.vec
                except:
                    gfu.vec.data = acd.mat.Inverse(freedofs=fes.FreeDofs(),inverse="sparsecholesky") * f.vec

                gfu = gfu.components[0] + sum([gfu.components[i+1]*self.config['enrich_functions'][i] for i in range(len(self.config['enrich_functions']))])
                
                error = sqrt (Integrate ((gfu-self.config['exact'])*(gfu-self.config['exact']), mesh))
                print ('Order:', order, 'Mesh Size:', size , "L2-error:", error)

        return 0
    
    
    '''Solves the convection-diffusion equations by the HDG 
    methods with the options to either enrich or not'''
    def _solveHDG(self):
        for order in self.config['order']:
            for size in self.config['mesh_size']:
                mesh = Mesh(unit_square.GenerateMesh(maxh=size))
                condense = True
                h = specialcf.mesh_size
                n = specialcf.normal(mesh.dim)
                dS = dx(element_boundary=True)

                V = L2(mesh, order=order)
                F = FacetFESpace(mesh, order=order, dirichlet=".*")
                Q = L2(mesh, order=0)
                QF = FacetFESpace(mesh, order=0)

                Vlist = [V]
                for enr_indicator in self.config['enrich_domain_ind']:
                    Vlist.append(mark_element(Q,mesh,enr_indicator,size))
                Vlist.append(F)
                for enr_indicator in self.config['enrich_domain_ind']:
                    Vlist.append(mark_element_bnd(Q, QF, mesh,enr_indicator, size))
                        
                fes = FESpace(Vlist) # [V, F, Qx, QFx, Qy, QFy])
                
                u = EnrichmentProxy_VOL(fes.TrialFunction(),self.config['enrich_functions'])
                v = EnrichmentProxy_VOL(fes.TestFunction(), self.config['enrich_functions'])

                uhat = EnrichmentProxy_FAC(fes.TrialFunction(),self.config['enrich_functions'])        
                vhat = EnrichmentProxy_FAC(fes.TestFunction(), self.config['enrich_functions'])

                print(type(u))
                print(type(uhat))

                jump_u = u-uhat
                jump_v = v-vhat

                print(type(jump_u))
                print(type(jump_v))

                # diffusion (Issue here)
                diffusion = grad(u) * grad(v) * dx + \
                    self.config['alpha'] * order ** 2/h * jump_u * jump_v * dS + \
                    (-grad(u) * n * jump_v - grad(v) * n * jump_u) * dS

                # convection            
                b = CoefficientFunction((self.config['beta'][0], self.config['beta'][1]))    
                
                uup = IfPos(b * n, u(), uhat)
                
                convection = -b * u * grad(v) * dx + b * n * uup * jump_v * dS

                acd = BilinearForm(fes, condense=condense)
                acd += self.config['epsilon'] *  diffusion + convection
                acd.Assemble()

                #rhs
                f = LinearForm(fes)
                f += SymbolicLFI(coeff * v)
                f.Assemble()

                gfu = GridFunction(fes)    
                if not condense:
                    inv = acd.mat.Inverse(fes.FreeDofs(), "umfpack")
                    gfu.vec.data = inv * f.vec
                else:
                    f.vec.data += acd.harmonic_extension_trans * f.vec

                    inv = acd.mat.Inverse(fes.FreeDofs(True), "umfpack")
                    gfu.vec.data = inv * f.vec

                    gfu.vec.data += acd.harmonic_extension * gfu.vec
                    gfu.vec.data += acd.inner_solve * f.vec
                    
                error = sqrt (Integrate ((gfu.components[0]-self.config['exact'])*(gfu.components[0]-self.config['exact']), mesh))
                print ('Order:', order, 'Mesh Size:', size , "L2-error:", error)

        return 0   

#### Setup and solve the convection diffusion problem

In [17]:
CT = Convection_Diffusion(new_config)
CT._solveHDG()

<class '__main__.EnrichmentProxy_VOL'>
<class '__main__.EnrichmentProxy_FAC'>
<class 'ngsolve.fem.CoefficientFunction'>
<class 'ngsolve.fem.CoefficientFunction'>


TypeError: unsupported operand type(s) for *: 'NoneType' and 'NoneType'